In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os


# Loading dataset 

In [16]:
path='./dataset/'
data=[]
labels=[]
class_list=os.listdir(path)
for c in class_list:
    dir_path=os.path.join(path,c)
    for i in os.listdir(dir_path):
        img_path=os.path.join(dir_path,i)
        image=cv2.imread(img_path)
        image=cv2.resize(image,(224,224))
        image=image/255.0
        data.append(image)
        if c=='fire_images':
            labels.append(0)
        elif c=='non_fire_images':
            labels.append(1)


In [19]:
print(len(data))
print(len(labels))

997
997


In [47]:
images=np.array(data)
lables=np.array(labels)

In [48]:
X_train,X_valid,Y_train,Y_valid=train_test_split(images, lables, test_size=0.20,random_state=0)

In [26]:
print(X_train.shape)
print(Y_train.shape)

(797, 224, 224, 3)
(200, 224, 224, 3)


# Model 

In [41]:
def create_custom_model(input_shape=(224,224,3), number_clases=2):
    model=models.Sequential()
    #Block1
    model.add(layers.Conv2D(16, kernel_size=(3,3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2,2)))
    #Block2
    model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2,2))
    
    #block3
    model.add(layers.Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2,2))
    
    #block4
    model.add(layers.Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(2,2))
    
    #block5
    
    model.add(layers.Flatten())
    model.add(layers.Dense(500, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(number_clases,activation='softmax'))
    
    return model

In [42]:
model=create_custom_model()

In [43]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 224, 224, 16)      448       
                                                                 
 conv2d_17 (Conv2D)          (None, 224, 224, 32)      4640      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 conv2d_18 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 conv2d_19 (Conv2D)          (None, 112, 112, 64)      36928     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                  

In [44]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [49]:
print(X_train.shape)
print(Y_train.shape)

(797, 224, 224, 3)
(797,)


In [50]:
histt=model.fit(X_train,Y_train, batch_size=8, epochs=20, validation_data=(X_valid,Y_valid))

Epoch 1/20
100/100 [==============================] - 78s 763ms/step - loss: 0.6379 - accuracy: 0.7265 - val_loss: 0.6101 - val_accuracy: 0.7500
Epoch 2/20
100/100 [==============================] - 77s 771ms/step - loss: 0.5766 - accuracy: 0.7553 - val_loss: 0.5742 - val_accuracy: 0.7500
Epoch 3/20
100/100 [==============================] - 75s 751ms/step - loss: 0.5160 - accuracy: 0.7616 - val_loss: 0.4273 - val_accuracy: 0.7950
Epoch 4/20
100/100 [==============================] - 77s 771ms/step - loss: 0.5154 - accuracy: 0.7553 - val_loss: 0.4458 - val_accuracy: 0.7500
Epoch 5/20
100/100 [==============================] - 79s 791ms/step - loss: 0.3878 - accuracy: 0.7867 - val_loss: 0.4917 - val_accuracy: 0.7500
Epoch 6/20
100/100 [==============================] - 75s 755ms/step - loss: 0.6731 - accuracy: 0.7553 - val_loss: 0.5474 - val_accuracy: 0.7650
Epoch 7/20
100/100 [==============================] - 75s 748ms/step - loss: 0.5734 - accuracy: 0.7478 - val_loss: 0.4824 - val_ac

In [51]:
model.save('fire_classification_model.h5')